<a href="https://colab.research.google.com/github/ihshin227/tensorlite_convert_test/blob/master/Keras_convlstm_convert_to_tensorflow_lite_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization

In [0]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1

                # Make it more robust by adding noise.
                # The idea is that if during inference,
                # the value of the pixel is not exactly one,
                # we need to train the network to be robust and still
                # consider it as a pixel belonging to a square.
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(15, 40, 40, 1),
                   padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
model.compile(loss='binary_crossentropy', optimizer=Adadelta())

In [0]:
noisy_movies, shifted_movies = generate_movies(n_samples=1200)
model.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10,
        epochs=3, validation_split=0.05)

Epoch 1/3
95/95 [==============================] - 33s 350ms/step - loss: 0.6927 - val_loss: 0.6923
Epoch 2/3
95/95 [==============================] - 32s 338ms/step - loss: 0.6919 - val_loss: 0.6915
Epoch 3/3
95/95 [==============================] - 32s 337ms/step - loss: 0.6911 - val_loss: 0.6907


In [0]:
from tensorflow.keras.models import save_model
#import tensorflow.keras.backend as K

#K.set_learning_phase(0)

filepath = './convlstm.h5'
tf.keras.models.save_model(model, filepath)

loaded = tf.keras.models.load_model(filepath, compile=False)

export_path = './convlstm'
loaded.save(export_path, save_format="tf")
#tf.saved_model.save(loaded, './convlstm')
#converter = tf.lite.TFLiteConverter.from_saved_model('./convlstm')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

In [0]:
print(noisy_movies[:1000].shape)
tflite_model = converter.convert()

(1000, 15, 40, 40, 1)


In [0]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))

RuntimeError: ignored

In [0]:
for layer in model.layers:
  print(tf.keras.layers.serialize(layer))
#tf.keras.layers.serialize(model.layers[0])

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

tflite_results = interpreter.get_tensor(output_details[0]['index'])
tf_results = model(tf.constant(input_data))

ConverterError: ignored